## Day 5, Part 2, Attempt 2

No more storing everything in a big old list

In [1]:
with open("./example.txt") as f:
    example_lines = [line.strip() for line in f.readlines()]

with open("./input.txt") as f:
    input_lines = [line.strip() for line in f.readlines()]

In [2]:
def get_seed_ranges(lines: list[str]) -> list[tuple[int]]:
    seeds_input = list(map(int, lines[0].split(": ")[-1].split()))
    seeds_ranges = []
    for i in range(0, len(seeds_input), 2):
        seeds_ranges.append(
            (seeds_input[i], seeds_input[i]+seeds_input[i+1])  # SO DOESN'T INCLUDE CLOSING VALUE i.e. behaves like Python Range
        )

    return seeds_ranges

In [31]:
def get_blocks(lines: list[str]) -> list[list[tuple[int, int, int]]]:
    # Ignore labels this time, just break the input up into blocks!
    blocks = []
    tmp = []
    for line in lines[3:]:
        if "map" in line:
            blocks.append(tmp)
            tmp = []
        elif not line:
            continue
        else:
            destination, source, length = list(map(int, line.split()))
            tmp.append(
                (destination, source, length)
            )
    blocks.append(tmp)  # oh my god how was I not adding THE LAST BLOCK????????
    
    return blocks

In [40]:
get_blocks(example_lines)[-1]  # that's right, we get the last block of mappings now. imagine not doing that

[(60, 56, 37), (56, 93, 4)]

Went a bit mad correcting the ranges manually and overthinking it. 

In [44]:
def part2(lines: list[str], verbose = True) -> int:
    seeds_ranges = get_seed_ranges(lines)
    
    # Now have a list of the seed ranges in form [(Xs, Xe), (Ys, Ye), ...] where s=start, e=end

    # Ignore labels this time, just break the input up into blocks!
    blocks = get_blocks(lines)

    # Now we have input blocks of the form
    # [map1=[(s1,d1,l1), (s2,d2,l2)], map2=[(s1,d1,l2)], ...]
    # So we can loop through these one at a time to tackle a mapping at a time.

    # Overlap types:
    # 1. none left, 2. overlap left, 
    # 3. overlap left right, 4. contained,
    # 5. overlap right, 6. none right
    # 7. perfect mapping

    for i, mappings in enumerate(blocks):
        if verbose: print(f"{i}/{len(blocks)}")
        if verbose: print(len(seeds_ranges))
        new_seeds_ranges = []
        while seeds_ranges:
            seed = seeds_ranges.pop()
            for destination, source, length in mappings:
                source_start, source_end = source, source+length - 1
                seed_start, seed_end = seed

                # 2. OVERLAP LEFT
                if source_start < seed_start and seed_start < source_end <= seed_end:
                    # so we can map from seed_start to source_end 
                    # but have to add source_end to seed_end to stack
                    seeds_ranges.append(
                        (source_end + 1, seed_end)
                    )
                    source_dest_diff = destination - source_start
                    new_seeds_ranges.append(
                        (seed_start + source_dest_diff, source_end + source_dest_diff)
                    )
                    break

                # 3. OVERLAP LEFT AND RIGHT
                elif source_start < seed_start and source_end > seed_end:
                    # seed is fully contained within the mappable range, so we
                    # can map it and not worry any further!

                    # JUST MAP THE MIDDLE
                    source_dest_diff = destination - source_start
                    new_seeds_ranges.append(
                        (seed_start + source_dest_diff, seed_end + source_dest_diff)
                    )
                    break

                # 4. COMPLETELY CONTAINED
                elif source_start >= seed_start and source_end <= seed_end:
                    # have to add the unmapped bits to left and right of seed to stack
                    seeds_ranges.append(
                        (seed_start, source_start - 1)
                    )
                    seeds_ranges.append(
                        (source_end + 1, seed_end)
                    )

                    # Map what we can of the source
                    source_dest_diff = destination - source_start
                    new_seeds_ranges.append(
                        (source_start + source_dest_diff, source_end + source_dest_diff)
                    )
                    break

                # 5. OVERLAP RIGHT
                elif seed_start <= source_start < seed_end and source_end > seed_end:
                    # can map from source_start to seed_end, but have to 
                    # add seed_start to source_start to the stack as it is unmapped
                    seeds_ranges.append(
                        (seed_start, source_start - 1)
                    )
                    source_dest_diff = destination - source_start
                    new_seeds_ranges.append(
                        (source_start + source_dest_diff, seed_end + source_dest_diff)
                    )
                    break
            else:
                # if we didn't go into an if/elif in the for loop it goes to else  (never found a mapping -> identity)
                new_seeds_ranges.append(seed)
        
        seeds_ranges = new_seeds_ranges

    if verbose: print(f"{len(blocks)}/{len(blocks)}")
    

    # Ok now it's quite dodgy I don't know where the 0s are coming from and why the bottom range is less than top range in some.......... this wasnt the case but not sure what I changed :)
    LHSs = [lower_seed for lower_seed, upper_seed in seeds_ranges if lower_seed != 0 and upper_seed > lower_seed]

    # Did fix these (0, -1) problems and (y,x) with y>x issues at one point but scrapped it when it still wasn't giving me right answer - so for now I settle for this

    return min(LHSs)

assert part2(example_lines, verbose=False) == 46
part2(input_lines)  # 96937421, 3294143, 3294142, 50007219 is WRONG don't submit it AGAIN   <- lol

0/7
10
1/7
32
2/7
57
3/7
97
4/7
124
5/7
185
6/7
217
7/7


69841803

In [46]:
def part2_the_pretty_version(lines: list[str], verbose = True) -> int:
    seeds_ranges = get_seed_ranges(lines)
    # Now have a list of the seed ranges in form [(Xs, Xe), (Ys, Ye), ...] where s=start, e=end+1

    blocks = get_blocks(lines)
    # Now we have input blocks of the form
    # [map1=[(d1,s1,l1), (d2,s2,l2)], map2=[(d1,s1,l2)], ...]
    # So we can loop through these one at a time to tackle a mapping at a time.

    for i, mappings in enumerate(blocks):
        if verbose: print(f"{i}/{len(blocks)}")
        if verbose: print(len(seeds_ranges))
        new_seeds_ranges = []
        while seeds_ranges:
            seed_start, seed_end = seeds_ranges.pop()
            for destination, source, length in mappings:
                # print(destination, source, length)
                overlap_start = max(seed_start, source)
                overlap_end = min(seed_end, source+length)
                # this range is non empty iff overlap_start < overlap_end
                if overlap_start < overlap_end:
                    new_start = overlap_start + (destination - source)
                    new_end = overlap_end + (destination - source)
                    new_seeds_ranges.append(
                        (new_start, new_end)
                    )
                    # Still now need to consider if we there were bits we didn't map!
                    if overlap_start > seed_start:
                        seeds_ranges.append(
                            (seed_start, overlap_start)
                        )
                    if overlap_end < seed_end:
                        seeds_ranges.append(
                            (overlap_end, seed_end)
                        )
                    break
                    # mapped and added to stack -> break!
            else:
                # No map available, so we do identity map, add to next batch
                new_seeds_ranges.append(
                    (seed_start, seed_end)
                )
        seeds_ranges = new_seeds_ranges

    if verbose: print(f"{len(blocks)}/{len(blocks)}")

    # print(sorted(seeds_ranges))
    return min(seeds_ranges)[0]

assert part2_the_pretty_version(example_lines, verbose=False) == 46
part2_the_pretty_version(input_lines)

0/7
10
1/7
21
2/7
33
3/7
53
4/7
66
5/7
97
6/7
113
7/7


69841803

In [48]:
if part2(input_lines, verbose=False) == part2_the_pretty_version(input_lines, verbose=False):
    print("free at last")

free at last


Yes! So clearly a few problems not solved w the edge cases etc. (although I'm fairly sure I fixed at some point then scrapped!) but I've spent long enough on this now, happy that the lowest 'LEGITIMATE' range was correct.

part2_the_pretty_version is an 'inspired' solution (I found it) that was structured similarly to mine and inspired actually fixing it/solving my reading in issue (and gave an answer to aim for!)
which was by https://www.youtube.com/watch?v=NmxHw_bHhGM  mr HyperNeutrino, some leaderboard guy.